In [256]:
import numpy as np
import math
import pandas as pd

### Standart MLP

In [257]:
def mean_squared_error(y_true, y_pred):
    return (np.square(y_true - y_pred)).mean(axis=None)

In [258]:
# Pesos inicializados com distribuição gaussiana de media 0 e variança 1
class MLP():
    def __init__(self, in_Length, hid_Length, out_Length, l_rate=0.002):

        self.W_in, self.B_in = self.initialize_weights(in_Length, hid_Length)
        self.W_out, self.B_out = self.initialize_weights(hid_Length, out_Length)

        self.l_rate = l_rate

        
    def initialize_weights(self, in_Length, hid_Length):
        w = np.random.random((hid_Length, in_Length))
        b = np.full((hid_Length), -1)

        return w, b

    def tanhFunc(self, x, deriv=False):
        if deriv:
            return 1 - np.power(np.tanh(x), 2)
        return  np.tanh(x)
    
    def sigmoidFunc(self, x, deriv=False):
        if deriv:
            sigmoid = self.sigmoidFunc(x)
            return sigmoid * (1 - sigmoid)
        return 1 / (1 + np.exp(-x))


    def forward(self, x):
        # O bias é adicionado após a ponderação e o dot product do vetor
        # self.out_hidden = np.dot(x.T, self.W_in) - self.B_in
        self.out_hidden = np.dot(self.W_in, x) + self.B_in
        self.out_hidden_activate = self.sigmoidFunc(self.out_hidden, deriv=False)

        self.output = np.dot(self.W_out, self.out_hidden_activate) + self.B_out
        y = self.sigmoidFunc(self.output, deriv=False)

        return y 
    
    def backward(self, target, x):

        # Gradient applied in last layer
        y = self.forward(x)
        
        Eout = y - target
        Ehin = self.sigmoidFunc(self.out_hidden, deriv=True) * np.dot(self.W_out, Eout)
        
        self.W_out = np.outer(self.out_hidden_activate, Eout)
        self.W_in = np.outer(x, Ehin)
        
        return y

In [259]:
mlp = MLP(3, 10, 1, 0.1)

In [260]:
train_data = pd.read_csv("../datasets/Apendice_3/train.csv")
test_data = pd.read_csv("../datasets/Apendice_3/test.csv")

In [261]:
def train(model, train_data, n_epochs, error_limit):
    batch_size = len(train_data) / 10

    train_x = train_data.values

    batches = np.array_split(train_x, batch_size)

    error_list = []
    for batch in batches:
        preds = []
        for row in batch:
            input = row[:-1]
            target = row[-1]

            y_pred = model.backward(target, input)
            preds.append(mean_squared_error(target, y_pred))

            if (len(error_list) != 0):
                error = (np.mean(np.array(preds)) - error_list[-1]) 
                error = error if error > 0 else (error * (-1))
                if  error < error_limit:
                    return error_list
        
        error_list.append(np.mean(np.array(preds)))

    return error_list

error = train(mlp, train_data, 10, 0.00001)

ValueError: shapes (1,10) and (1,) not aligned: 10 (dim 1) != 1 (dim 0)

In [ ]:
accuracy = 0.0

for batch in test_data.values:
    output = mlp.forward(batch[:-1])
    if output == batch[-1]:
        accuracy += 1
        
accuracy /= len(test_data)
print(accuracy)

0.0
